In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-8c74f297-f026-484d-93d3-c048f8e2b386"
os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-f5823dc5-753d-4db7-a052-0bf0cfe9c3cf"
os.environ["LANGFUSE_HOST"] = "https://cloud.langfuse.com"
os.environ["PERPLEXITY"] = "False"

In [3]:
from anthropic import AnthropicBedrock
from application import Application
from compiler.core import Compiler
from core.interpolator import Interpolator

In [4]:
compiler = Compiler("botbuild/tsp_compiler", "botbuild/app_schema")
client = AnthropicBedrock(aws_profile="dev", aws_region="us-west-2")

In [5]:
application = Application(client, compiler)
interpolator = Interpolator(".")

In [6]:
application_description = """
Bot that does web search
""".strip()

In [7]:
application_description = "Bot that greets user by name and returns personalized greeting."

In [8]:
my_bot = application.create_bot(application_description, bot_id="handler_tests")

Skipping Initial Description Refinement
Compiling TypeSpec...
Compiling Typescript Schema Definitions...
Compiling Drizzle...
Compiling Handler Tests...
Compiling Handlers...


In [9]:
interpolator.bake(my_bot, "../../bot_result") # next to repo root

In [10]:
my_bot.__dict__

{'refined_description': RefineOut(refined_description='Bot that greets user by name and returns personalized greeting.', error_output=None),
 'typespec': TypespecOut(reasoning='I expect user to send messages like "Hello, my name is John" or "Hi, I\'m Jane".\nLLM can extract the name from the message and pass it to the greeting handler.\n"Hello, my name is John" -> greet({name: "John"})\nThe bot should be able to:\n- Store and retrieve user\'s name\n- Generate personalized greeting\n- Keep track of greeting count', typespec_definitions='model GreetingRequest {\n    name: string;\n    preferredLanguage: string;\n}\n\nmodel UserStats {\n    name: string;\n    greetingCount: integer;\n    firstGreetedAt: utcDateTime;\n    lastGreetedAt: utcDateTime;\n}\n\ninterface GreetingBot {\n    @llm_func("Generate personalized greeting for user")\n    greet(options: GreetingRequest): string;\n\n    @llm_func("Get user\'s greeting statistics")\n    getUserStats(options: GreetingRequest): UserStats;\n}